In [ ]:
import os

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
try:
    import spnspecs

    spnspecs.set_graph_specifications()
except:
    spnspecs = None

In [ ]:
figpth = "../Figures"
width = 6.8
dpi = 300

In [ ]:
def no_smoothing(sat):
    f = sat.copy()
    f[sat < 0] = 0.0
    f[sat >= 0] = 1.0
    return f

In [ ]:
def linear_smoothing(sat):
    f = sat.copy()
    f[sat < 0] = 0.0
    f[sat > 1] = 1.0
    return f

In [ ]:
def cubic_smoothing(sat, c1=-1, c2=2):
    f = c1 * sat**3 + c2 * sat**2
    f[sat < 0] = 0
    f[sat > 1] = 1
    return f

In [ ]:
def cubic_smoothing_dervsat(sat, c1=-1, c2=2):
    dfdx = 3 * c1 * sat**2 + 2.0 * c2 * sat
    dfdx[sat < 0] = 0
    dfdx[sat > 1] = 0
    return dfdx

In [ ]:
def cubic_smoothingalt(x, xrange, c1=-1, c2=2):
    sat = x / xrange
    cof1 = c1 / (xrange) ** 3
    cof2 = c2 / (xrange) ** 2
    f = cof1 * x**3 + cof2 * x**2
    f[sat < 0] = 0
    f[sat > 1] = 1
    return f

In [ ]:
def cubic_smoothing_dervh(x, xrange, c1=-1, c2=2):
    sat = x / xrange
    cof1 = 3 * c1 / (xrange) ** 3
    cof2 = 2 * c2 / (xrange) ** 2
    dfdx = cof1 * x**2 + cof2 * x
    dfdx[sat < 0] = 0
    dfdx[sat > 1] = 0
    return dfdx

In [ ]:
def fd_derv(x, v):
    isize = v.shape[0]
    derv = np.ones((isize), dtype=v.dtype) * np.nan
    for i in range(1, isize - 2):
        dv = v[i + 1] - v[i - 1]
        dx = x[i + 1] - x[i - 1]
        derv[i] = dv / dx
    return derv

In [ ]:
dtype = np.float
step = 0.001
s0, s1 = -0.1, 1.1
s = np.arange(s0, s1, step=step, dtype=dtype)
sp = s.copy()
sp[s < 0] = 0.0
sp[s > 1] = 1.0

In [ ]:
s1, s.shape

In [ ]:
plt.plot(s, sp * sp)

In [ ]:
f0 = cubic_smoothing(s)
f1 = cubic_smoothing(s, c1=-2.0, c2=3.0)

In [ ]:
dseep = 1.0
z0 = 0.0
xdiff = dseep * s
h = xdiff + z0
dsdh = 1.0 / dseep

In [ ]:
plt.plot(s, sp, lw=1.5, color="black", label="Linear")
plt.plot(s, f0, lw=0.75, color="blue", label="GWSEEP")
plt.plot(s, f1, lw=0.75, color="red", label="WELL")
if spnspecs is not None:
    spnspecs.graph_legend(plt.gca())
# plt.xlim(0.9, 1.1)
# plt.ylim(0.9, 1.1)

In [ ]:
f0p = cubic_smoothingalt(xdiff, dseep)
f1p = cubic_smoothingalt(xdiff, dseep, c1=-2.0, c2=3.0)

In [ ]:
plt.plot(s, f0, lw=0.75, color="blue", label="GWSEEP")
plt.plot(s, f0p, lw=0.0, color="blue", marker="o", mfc="none", ms=4, markevery=50)
plt.plot(s, f1, lw=0.75, color="red", label="WELL")
plt.plot(s, f1p, lw=0.0, color="red", marker="o", mfc="none", ms=4, markevery=50)

In [ ]:
plt.plot(s, f0 - f0p, lw=0.75, color="blue", label="GWSEEP")
plt.plot(s, f1 - f1p, lw=0.75, color="red", label="WELL")

In [ ]:
spderv = np.ones(s.shape, dtype=np.float)
spderv[s < 0] = 0.0
spderv[s > 1] = 0.0

In [ ]:
f0derv = cubic_smoothing_dervsat(s)
f1derv = cubic_smoothing_dervsat(s, c1=-2.0, c2=3.0)

In [ ]:
f0derv2 = cubic_smoothing_dervh(xdiff, dseep)
f1derv2 = cubic_smoothing_dervh(xdiff, dseep, c1=-2.0, c2=3.0)

In [ ]:
plt.plot(s, f0derv, lw=4, color="blue", label="GWSEEP", alpha=0.5)
plt.plot(
    s, f0derv2 / dsdh, lw=0.75, color="black", ls="-."
)  # marker='o', mfc='none', ms=4, markevery=50)
plt.plot(s, f1derv, lw=4, color="red", label="WELL", alpha=0.5)
plt.plot(
    s, f1derv2 / dsdh, lw=0.75, color="black", ls="-."
)  # marker='o', mfc='none', ms=4, markevery=50)

In [ ]:
plt.plot(s, fd_derv(h, f0), lw=0.75, color="blue", label="GWSEEP")
plt.plot(s, fd_derv(h, f1), lw=0.75, color="red", label="WELL")
plt.ylabel(r"$\frac{\partial F}{\partial h}$");

In [ ]:
plt.plot(s, fd_derv(h, f0p), lw=0.75, color="blue", label="GWSEEP")
plt.plot(s, fd_derv(h, f1p), lw=0.75, color="red", label="WELL")
plt.ylabel(r"$\frac{\partial F}{\partial h}$");

### Plot data

In [ ]:
fig, axes = plt.subplots(
    nrows=1, ncols=2, tight_layout=True, figsize=(width, (1.4 / s1) * width / 2)
)

letters = ["A", "B"]
for idx, ax in enumerate(axes):
    ax.set_xlim(s0, s1)
    ax.set_ylim(s0, 1.4)
    ax.set_xticks([0, 0.25, 0.5, 0.75, 1])
    ax.set_xticklabels(["0", "0.25", "0.50", "0.75", "1.00"])
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1, 1.25])
    ax.set_yticklabels(["0", "0.25", "0.50", "0.75", "1.00", "1.25"])
    if spnspecs is not None:
        spnspecs.remove_edge_ticks(ax)
        spnspecs.heading(ax, letter=letters[idx])

ax = axes[0]
ax.axhline(0, lw=0.5, ls="-.", color="black")
ax.axhline(1, lw=0.5, ls="-.", color="black")
ax.axvline(0, lw=0.5, ls="-.", color="black")
ax.axvline(1, lw=0.5, ls="-.", color="black")
ax.plot(s, sp, lw=3.5, color="blue", label=r"$F_{DRN}$")
ax.plot(s, f0, lw=1.75, color="red", label=r"$F_{DRN}^*$")
ax.set_xlabel(r"$\frac{h - ZDRN}{DDRN}$, unitless")
ax.set_ylabel("Discharge scale factor, unitless")

ax = axes[1]
ax.axhline(0, lw=0.5, ls="-.", color="black")
ax.axhline(1, lw=0.5, ls="-.", color="black")
ax.axvline(0, lw=0.5, ls="-.", color="black")
ax.axvline(1, lw=0.5, ls="-.", color="black")
ax.plot([0, 1], [1, 1], lw=3.5, color="blue", label=r"$F_{DRN}$ (linear)")
ax.plot([s0, 0], [0, 0], lw=3.5, color="blue", label=None)
ax.plot([1, s1], [0, 0], lw=3.5, color="blue", label=None)
ax.plot(s[s <= 1], f0derv[s <= 1], lw=1.75, color="red", label=r"$F_{DRN}^*$ (cubic)")
ax.plot([1, s1], [0, 0], lw=1.75, color="red", label=None)
if spnspecs is not None:
    spnspecs.graph_legend(ax, loc="lower right", bbox_to_anchor=(0.9, 0.05))
ax.set_xlabel(r"$\frac{h - ZDRN}{DDRN}$, unitless")
ax.set_ylabel(
    "Discharge scale factor derivative with respect to "
    + r"$\frac{h - ZDRN}{DDRN}$, unitless"
)

fpth = os.path.join(figpth, "DischargeScaleFactor.pdf")
fig.savefig(fpth, dpi=dpi)

In [ ]:
dc = np.arange(0, 1, 0.0001)
z = np.zeros(dc.shape, dtype=np.float)
slope = 0.2
dadd = 0.1
d = dadd
for idx in range(1, dc.shape[0]):
    v = dc[idx]
    if v > d:
        slope *= -1.0
        d += dadd
    z[idx] = z[idx - 1] + slope * (v - dc[idx - 1])
z -= z.max() / 2
zmin, zmax = z.min(), z.max()
ymin, ymax = -slope / 10, slope / 10

In [ ]:
t = np.arange(s.shape[0]) / s.shape[0]
xdiffq = xdiff.copy()
xdiffq[xdiff < 0] = 0
q0 = no_smoothing(s) * xdiffq
ql = linear_smoothing(s) * xdiffq
qc = cubic_smoothing(s) * xdiffq

In [ ]:
arrowprops = dict(arrowstyle="->", connectionstyle="arc3")

In [ ]:
fig = plt.figure(tight_layout=True, figsize=(width, (2 / 3) * width))
gs = gridspec.GridSpec(2, 3)
axes = [
    fig.add_subplot(gs[0, :]),
    fig.add_subplot(gs[1, 0]),
    fig.add_subplot(gs[1, 1]),
    fig.add_subplot(gs[1, 2]),
]
xlims = [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]
ylims = [(ymax, ymin), (s0, s1), (0, s1), (0, 600)]

letters = ["A", "B", "C", "D"]
for idx, ax in enumerate(axes):
    ax.set_xlim(xlims[idx])
    ax.set_ylim(ylims[idx])
    if idx > 0:
        ax.set_xticks([0, 0.25, 0.5, 0.75, 1])
        ax.set_xticklabels(["0", "0.25", "0.50", "0.75", "1.00"])
    else:
        ax.tick_params(axis="both", length=0, labelbottom=False, labelleft=False)
    if spnspecs is not None:
        spnspecs.remove_edge_ticks(ax)
        spnspecs.heading(ax, letter=letters[idx])

ax = axes[0]
ax.fill_between(dc, ymax, 0, color="0.9")
# ax.plot(dc, z, lw=1, color='black')
ax.axhline(zmin, lw=0.5, ls="--", color="black")
ax.axhline(zmax, lw=0.5, ls="--", color="black")
ax.axhline(0, lw=0.75, color="black")
if spnspecs is not None:
    text = "DDRN"
    # spnspecs.add_text(ax, 'DDRN', x=1.01, y=0.5, ha='center', va='center',
    #                  bold=False, rotation=-90)
    spnspecs.add_annotation(
        ax,
        text=text,
        xy=(1.01, 0.77),
        xytext=(1.01, 0.5),
        bold=False,
        rotation=-90,
        ha="center",
        va="center",
        xycoords="axes fraction",
        textcoords="axes fraction",
        arrowprops=arrowprops,
    )
    spnspecs.add_annotation(
        ax,
        text=text,
        xy=(1.01, 0.23),
        xytext=(1.01, 0.5),
        bold=False,
        rotation=-90,
        ha="center",
        va="center",
        xycoords="axes fraction",
        textcoords="axes fraction",
        arrowprops=arrowprops,
    )
    text = "Land surface elevation"
    #     spnspecs.add_annotation(ax, text=text, xy=(0.6, 0.48), xytext=(0.68, 0.9),
    #                             bold=False, ha='center', va='center',
    #                             xycoords='axes fraction', textcoords='axes fraction',
    #                             arrowprops=arrowprops)
    spnspecs.add_text(ax, text=text, x=0.5, y=0.5, bold=False, ha="center", va="bottom")
    text = "Cell top"
    spnspecs.add_text(ax, text=text, x=0.01, y=0.5, bold=False, ha="left", va="bottom")
    text = "Cell bottom"
    spnspecs.add_text(ax, text=text, x=0.01, y=0.01, bold=False, ha="left", va="bottom")
    text = r"Land surface elevation + $\frac{DDRN}{2}$"
    spnspecs.add_text(
        ax, text=text, x=0.5, y=zmax, transform=False, bold=False, ha="center"
    )
    text = r"Land surface elevation - $\frac{DDRN}{2}$"
    spnspecs.add_text(
        ax, text=text, x=0.5, y=zmin, transform=False, bold=False, ha="center", va="top"
    )
    text = r"$ZDRN + DDRN$"
    spnspecs.add_text(
        ax, text=text, x=0.99, y=zmax, transform=False, bold=False, ha="right"
    )
    text = r"$ZDRN$"
    spnspecs.add_text(
        ax, text=text, x=0.99, y=zmin, transform=False, bold=False, ha="right", va="top"
    )

ax = axes[1]
ax.axhline(0, lw=0.5, ls="-.", color="black")
ax.axhline(1, lw=0.5, ls="-.", color="black")
ax.plot(t, s, lw=1.5, color="0.5", ls="--", label=r"$F_{DRN}$")
ax.set_xlabel("Fractional simulation time, unitless")
ax.set_ylabel(r"$\frac{h - ZDRN}{DDRN}$, unitless")
text = r"$h = ZDRN + DDRN$"
spnspecs.add_text(
    ax, text=text, x=0.02, y=0.98, transform=False, bold=False, ha="left", va="top"
)
text = r"$h = ZDRN$"
spnspecs.add_text(
    ax, text=text, x=0.99, y=0.01, transform=False, bold=False, ha="right", va="bottom"
)


ax = axes[2]
ax.plot(t, q0, lw=0.75, color="black")
ax.plot(t, ql, lw=1.5, color="blue")
ax.plot(t, qc, lw=0.75, color="red", label=r"$F_{DRN}^*$")
ax.set_xlabel("Fractional simulation time, unitless")
ax.set_ylabel(r"Drain discharge rate, , L$^3$/T")
text = r"Area = 1 L$^2$" + "\n" + r"K$_v$ = 1 L/T"
if spnspecs is not None:
    spnspecs.add_text(
        ax, text=text, x=0.1, y=0.93, italic=False, bold=False, ha="left", va="top"
    )

ax = axes[3]
ax.plot(t, q0.cumsum(), lw=0.75, color="black", label=r"$F_{DRN}^0$ (original)")
ax.plot(t, ql.cumsum(), lw=1.5, color="blue", label=r"$F_{DRN}$ (linear)")
ax.plot(t, qc.cumsum(), lw=0.75, color="red", label=r"$F_{DRN}^*$ (cubic)")
ax.set_xlabel("Fractional simulation time, unitless")
ax.set_ylabel(r"Cumulative drain discharge, L$^3$")
if spnspecs is not None:
    spnspecs.graph_legend(ax, loc="upper left", labelspacing=0.15)

fpth = os.path.join(figpth, "DRNDischargeDifferences.pdf")
fig.savefig(fpth, dpi=dpi)

In [ ]:
q0.max(), ql.max(), qc.max()

In [ ]:
q0.cumsum().max(), ql.cumsum().max(), qc.cumsum().max()